# Diabetic analysis & modeling
## 1. Exploratory Data Analysis

In [155]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [156]:
df = pd.read_csv("../data/diabetic_data.csv")

In [157]:
df.replace("?", np.nan, inplace=True)

In [ ]:
df.readmitted.value_counts()

- There are 50 columns and more than 100k rows for the diabetic data
- The target label is **readmitted** which is category with 3 labels: 
    - "No": No readmission for inpatient
    - ">30": patient was readmitted in 30 days or more
    - "<30": patient was readmitted in less than 30 days
- There are total 48 input variables

In [159]:
df = df.drop(["encounter_id","patient_nbr"],axis=1)

### 1.1. Missing values & Imputation
#### 1.1.1. Missing values

In [160]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
missing_pct = missing_values / len(df) * 100


In [ ]:
missing_pct.index

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
missing_pct.plot(kind='bar', ax=ax)
for idx,vals in enumerate(missing_pct):
    ax.text(idx, vals, f"{vals:.2f}%", ha='center', va='bottom')
ax.set_title('Percentage of Missing Values by Column')
ax.set_ylabel('Percentage (%)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()


In [ ]:
df.medical_specialty.value_counts(normalize=True)*100

- Out of 48 input variables, 9 variables have missing values
- **Weight** is a numeric variable and has 97% missing value. Although it is important feature, but due to lots of missing values, we have to ignore this variable.
- Similarly, we remove **max_glu_serum and A1Cresult** due to major missing values
- Medical_Specialty containts information for the treatment procedure. It has about 49% missing values. According to suggestion, we will convert the missing value to None and apply OHE to this.
- Similarly, we will apply the same for **payer_code** and **race**
- diag1, diag2, diag3 will be combined and imputed in the later part

#### 1.1.2. Imputations
- First remove **weight** column

In [164]:
df = df.drop(["weight","max_glu_serum","A1Cresult"],axis=1)

- Next, impute **medical_specialty** with **None**
- To reduce the dimension of this variable, we merge all categorical with less than 1% occurrence to the new variable called **Rest**


In [165]:
df.medical_specialty.fillna("None",inplace=True)
small_specialty = df.medical_specialty.value_counts(normalize=True)*100
df.medical_specialty = df.medical_specialty.replace(small_specialty.index[small_specialty<=1].values,"Rest")

- Impute **payer_code**

In [166]:
df.payer_code.fillna("None",inplace=True)
small_pcode = df.payer_code.value_counts(normalize=True)*100
df.payer_code = df.payer_code.replace(small_pcode.index[small_pcode<=1].values,"Rest")

- Impute **race**
- We do not merge any other race for this varialbe due to very small amount of missing value

In [167]:
df.race.fillna("None",inplace=True)

- Impute **diag1, diag2, diag3**: replace Nan value with **None**
- For each diag category, retain the top 1% category value (~top 20 category), the rest convert to **Rest**
- Create new diag column with following criteria:
    - if there are values in diag_1, diag_2 and diag_3: get the value from diag_1 only
    - if there are values in diag_2 and diag_3 only: get the value from diag_2 only
- Keep only diag column and remove the rest    




In [168]:
df.diag_1.fillna("None",inplace=True)
df.diag_2.fillna("None",inplace=True)
df.diag_3.fillna("None",inplace=True)

In [169]:
ind1 = df.diag_1.value_counts(normalize=True)*100
df.diag_1 = df.diag_1.replace(ind1.index[ind1<=1].values,"Rest")

ind2 = df.diag_2.value_counts(normalize=True)*100
df.diag_2 = df.diag_2.replace(ind2.index[ind2<=1].values,"Rest")

ind3 = df.diag_3.value_counts(normalize=True)*100
df.diag_3 = df.diag_3.replace(ind3.index[ind3<=1].values,"Rest")

In [170]:
def pick_diag(row):
    for col in ['diag_1', 'diag_2', 'diag_3']:
        val = row[col]
        if pd.notna(val) and val != 'Rest' and val != 'None':
            return val
    return 'Rest'  # If all are "Rest" or None

# Apply to DataFrame
df['diag'] = df.apply(pick_diag, axis=1)

In [171]:
df = df.drop(["diag_1","diag_2","diag_3"],axis=1)

- Convert **age** from category to numeric

In [172]:
age_map = {
    '[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35,
    '[40-50)': 45, '[50-60)': 55, '[60-70)': 65, '[70-80)': 75,
    '[80-90)': 85, '[90-100)': 95
}
df['age_mid'] = df['age'].map(age_map)
df = df.drop(["age"],axis=1)


### 1.2. Catogorical and Numerical data analysis
- Beside the object type categorical variables, there are 3 columns having categorical values in numeric format that need to be treated as categorical variables, they are **admission_type_id, discharge_disposition_id, admission_source_id**

In [189]:
cat_cols = df.select_dtypes("object").columns
df_cat = pd.concat([df[cat_cols],df[["admission_type_id","discharge_disposition_id","admission_source_id"]]],axis=1)

In [191]:
df_num = df.drop(df_cat.columns,axis=1)

df_cat = df_cat.drop(["readmitted"],axis=1)

In [ ]:
df_cat.shape

#### 1.2.1. Categorical distribution plots

In [ ]:
def plot_categorical_grid(df, cat_cols, target_col='readmitted', n_rows=11, n_cols=3):
    # Set up figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
    axes = axes.flatten()

    # Define color mapping
    expected_order = ["NO", "<30", ">30"]
    colors = {"NO": "green", "<30": "blue", ">30": "red"}

    for i, col in enumerate(cat_cols):
        ax = axes[i]
        
        # Build count DataFrame
        count_df = (
            df.groupby([col, target_col])
            .size()
            .reset_index(name='count')
            .pivot(index=col, columns=target_col, values='count')
            .fillna(0)
        )

        # Sort by total count descending
        count_df['total'] = count_df.sum(axis=1)
        count_df = count_df.sort_values(by='total', ascending=False).drop(columns='total')

        # Normalize to get proportions
        total_count = count_df.values.sum()
        prop_df = count_df / total_count * 100

        # Reorder columns to expected order
        available_order = [t for t in expected_order if t in prop_df.columns]
        prop_df = prop_df[available_order]

        # Plot
        prop_df.plot(
            kind='bar',
            stacked=True,
            ax=ax,
            color=[colors[c] for c in available_order],
            legend=False
        )

        ax.set_title(col)
        ax.set_ylabel("Proportion (%)")
        ax.set_xlabel("")
        ax.tick_params(axis='x', rotation=45)

    # Hide unused axes
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    # Shared legend
    handles = [plt.Rectangle((0,0),1,1,color=colors[k]) for k in expected_order if k in df[target_col].unique()]
    labels = [k for k in expected_order if k in df[target_col].unique()]
    fig.legend(handles, labels, title=target_col, loc='upper center', ncol=3)

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

plot_categorical_grid(df, df_cat.columns)

From the categorical plots we can see that: 
- **readmitted**: is the target variables and was plotted against all other categorical input data. The distribution of NO readmittion covers majority followed by ">30" days value
- **race**: more than 70% of patients are caucasian, this having good pattern in comparison to target variable
- Some good distributions that should be retained are: **gender, payer_code, medical_Specialty, metformin, glipizide, glyburide, piolitazone, rosiglitazone, insulin, change, diabetesMed, diag, admission_type_id, discharge_disposition_id, admission_source_id**.
- **admission_type_id**: more than 50% are type 1, which is Emergency intake, while Urgent and Elective are highly ranked behind
- **discharge_disposition_id**: about 60% of diabetic discharge are to home, while 13% are discharge to SNF or home with home health service. So we can see that majority discharge are to home
- **admission_source_id**: similar to admission_type, nearly 60% admission sources are from 7: Emergency room, followed by 30% of 1:Physician Referral
- The following categorical variables are suggested to be removed due to extreme skewness: **repalinize, nateglinide, chloropropamite, acetohesamide, tolbutamide, acarbose, miglitol, troglitazone, tolazamide, examide, citoglipton, gliburide_metoformin, glipizide_metoformin, gplipiride_glitopetazone, troglitazone, metorformin_piglitazone**

In [196]:
df_cat_keep = df[["race","gender","payer_code","medical_specialty","metformin","glimepiride","glipizide","glyburide","pioglitazone",
                  "rosiglitazone","insulin","change","diabetesMed","diag","admission_type_id", "discharge_disposition_id", "admission_source_id"]]

#### 1.2.2. Numerical data analysis

In [198]:
corr = df_num.corr(method="pearson")

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True, linewidths=0.5)
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()

- Cross correlation matrix does not exhibit high collinearity between numeric variables
- Next we compute VIF to check if there is any significant collinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data['Feature'] = df_num.columns
vif_data['VIF'] = [variance_inflation_factor(df_num.values, i) for i in range(df_num.shape[1])]


In [ ]:
vif_data

- The VIF analysis shows potential high collinearity between number_dianoses and age, however, the VIF values are in acceptable range. Hence we do not remove anything
- We will proceed to One Hot Encoding and data partitioning
### 1.3. One Hot Encoding categorical values
- There are 33 original categorical variables
- After EDA, we narrow down to 17 categorical variables
- Now, we apply One Hot Encoding to 17 variables and we have 153 new variables
- Merge this new OHE dataframe with numerical data to have X dataframe



In [ ]:
df_cat_keep = df_cat_keep.astype(str)

In [217]:
df_cat_ohe = pd.get_dummies(df_cat_keep,drop_first=False,dtype=int)

In [ ]:
df_cat_ohe.shape

In [220]:
X = pd.concat([df_cat_ohe,df_num],axis=1)

In [ ]:
y = df["readmitted"]

### 1.4. Partition X and y to training and testing set

In [238]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=123)

## 2. Modeling
### 2.1. Logistic Regression
#### Perform K-Fold Cross Validation

In [228]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [227]:
kf = KFold(n_splits=10,shuffle=True,random_state=123)

In [251]:
pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegressionCV(
        Cs=10,                     # Number or list of regularization values to try
        cv=10,                      # Number of cross-validation folds
        multi_class='multinomial',# For softmax-style multi-class
        solver='lbfgs',           # Good for multiclass
        max_iter=1000,
        scoring='accuracy',
        random_state=123
    )
)

In [ ]:
pipeline.fit(X_train, y_train)


In [253]:
y_pred = pipeline.predict(X_test)


In [ ]:
# Best C chosen for each class
model = pipeline.named_steps['logisticregressioncv']
print("Best C per class:", model.C_)

# Accuracy
from sklearn.metrics import accuracy_score
print("Test Accuracy:", accuracy_score(y_test, y_pred))
